In [1]:
import nltk
import pickle
import wikipedia
import pandas as pd
import numpy as np
from statistics import mean

stop_words = set(nltk.corpus.stopwords.words('english'))

import import_ipynb
import wiktionary_parser
import preprocess_utils

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Phrases
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

from semantic_text_similarity.models import ClinicalBertSimilarity

importing Jupyter notebook from wiktionary_parser.ipynb
importing Jupyter notebook from preprocess_utils.ipynb


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


importing Jupyter notebook from bert_semanitic_similarity_experiments.ipynb


In [ ]:
clinical_model = ClinicalBertSimilarity(device='cuda', batch_size=10)

In [37]:
def title_domain_predict(curtext, domain_arr):
    
    domains = []
    for arr in domain_arr:
        temp = []
        for word in arr:
            if word not in stop_words and word!=',':
                temp.append(word)
        domains.append(temp)

    title_tokens = nltk.word_tokenize(title.lower())

    
    FINAL = []
    for dt in domain_arr:
        temp2 = []
        for dtt in dt:
            temp1 = []
            for tt in title_tokens:
                try:
                    similarity = glove_vectors.similarity(dtt,tt)
                except:
                    similarity = 0
                        
                temp1.append(similarity)
            temp2.append(np.mean(temp1))
        FINAL.append(np.mean(temp2))
    return FINAL

In [38]:
def semantic_predict(curtext, clean_defs):
    final = []
    for i in clean_defs:
        temp = []
        for j in nltk.sent_tokenize(curtext):
                temp.append(clinical_model.predict([(j,i)]))
        final.append(np.mean(temp))
    return final

In [39]:
def embedded_driver(curtext, kw, pos):
    orig_defs =  [obj['def'] for obj in wiktionary_parser.define(kw)[pos]]
    domain_arr = [preprocess_utils.clean_text(sent[:sent.find(')')],False,False,True,True,True).replace('.','').replace('(','').strip().split(', ')for sent in orig_defs]
    clean_defs = [preprocess_utils.clean_text(definition,False,True,True,True,True) for definition in orig_defs]
    
    predict1_probs = title_domain_predict(curtext, domain_arr)
    
    predict2_probs = semantic_predict(curtext, clean_defs)
        
    return predict1_probs, predict1_probs

In [40]:
embedded_driver('computer', 'short', 'adjective')

NameError: name 'title' is not defined

In [ ]:
# def main_driver()